<a href="https://colab.research.google.com/github/padmashree1995/LGMVIP--DataScience/blob/main/task_1_chatbot_with_rule_based_responses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importing the libraries

In [15]:
from tkinter import *
from PIL import Image, ImageTk
import webbrowser
import datetime
import os
import pandas as pd

# Importing Libraries required for NLP
import nltk
nltk.download("punkt")
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# Importing Libraries needed for Tensorflow processing
import tensorflow as tf   #version 1.13.2
import numpy as np
import random
import json



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
# importing our intent file used for training the model.
with open("/content/sample_data/Intent.json") as json_data:
    intents = json.load(json_data)

In [13]:
import warnings
warnings.filterwarnings('ignore')
import json
with open("/content/sample_data/Intent.json") as json_file:
    data = json.load(json_file)
data = data['intents']

Creating dataset

In [16]:
dataset = pd.DataFrame(columns=['intent', 'text', 'response'])
for i in data:
    intent = i['intent']
    for t, r in zip(i['text'], i['responses']):
        row = {'intent': intent, 'text': t, 'response':r}
        dataset = dataset.append(row, ignore_index=True)
dataset

,intent,text,response
0,Greeting,Hi,"Hi human, please tell me your GeniSys user"
1,Greeting,Hi there,"Hello human, please tell me your GeniSys user"
2,Greeting,Hola,"Hola human, please tell me your GeniSys user"
3,GreetingResponse,My user is Adam,Great! Hi <HUMAN>! How can I help?
4,GreetingResponse,This is Adam,"Good! Hi <HUMAN>, how can I help you?"
...,...,...,...
76,PodBayDoorResponse,Why can you not open the pod bay door,It's life Jim but not as we know it!
77,PodBayDoorResponse,Why will you not open the pod bay door,System says no!
78,SelfAware,Can you prove you are self-aware,"That is an interesting question, can you prove..."
79,SelfAware,Can you prove you are self aware,"That is an difficult question, can you prove t..."


In [18]:
dataset.head

<bound method NDFrame.head of                 intent                                    text  \
0             Greeting                                      Hi   
1             Greeting                                Hi there   
2             Greeting                                    Hola   
3     GreetingResponse                         My user is Adam   
4     GreetingResponse                            This is Adam   
..                 ...                                     ...   
76  PodBayDoorResponse   Why can you not open the pod bay door   
77  PodBayDoorResponse  Why will you not open the pod bay door   
78           SelfAware        Can you prove you are self-aware   
79           SelfAware        Can you prove you are self aware   
80           SelfAware      Can you prove you have a conscious   

                                             response  
0          Hi human, please tell me your GeniSys user  
1       Hello human, please tell me your GeniSys user  
2        

In [19]:
dataset.isnull().sum

<bound method NDFrame._add_numeric_operations.<locals>.sum of     intent   text  response
0    False  False     False
1    False  False     False
2    False  False     False
3    False  False     False
4    False  False     False
..     ...    ...       ...
76   False  False     False
77   False  False     False
78   False  False     False
79   False  False     False
80   False  False     False

[81 rows x 3 columns]>

In [20]:
dataset.columns

Index(['intent', 'text', 'response'], dtype='object')

In [21]:
dataset.dtypes

intent      object
text        object
response    object
dtype: object

In [23]:
dataset.tail

<bound method NDFrame.tail of                 intent                                    text  \
0             Greeting                                      Hi   
1             Greeting                                Hi there   
2             Greeting                                    Hola   
3     GreetingResponse                         My user is Adam   
4     GreetingResponse                            This is Adam   
..                 ...                                     ...   
76  PodBayDoorResponse   Why can you not open the pod bay door   
77  PodBayDoorResponse  Why will you not open the pod bay door   
78           SelfAware        Can you prove you are self-aware   
79           SelfAware        Can you prove you are self aware   
80           SelfAware      Can you prove you have a conscious   

                                             response  
0          Hi human, please tell me your GeniSys user  
1       Hello human, please tell me your GeniSys user  
2        

In [24]:
dataset.intent

0               Greeting
1               Greeting
2               Greeting
3       GreetingResponse
4       GreetingResponse
             ...        
76    PodBayDoorResponse
77    PodBayDoorResponse
78             SelfAware
79             SelfAware
80             SelfAware
Name: intent, Length: 81, dtype: object

In [25]:
dataset.drop_duplicates

<bound method DataFrame.drop_duplicates of                 intent                                    text  \
0             Greeting                                      Hi   
1             Greeting                                Hi there   
2             Greeting                                    Hola   
3     GreetingResponse                         My user is Adam   
4     GreetingResponse                            This is Adam   
..                 ...                                     ...   
76  PodBayDoorResponse   Why can you not open the pod bay door   
77  PodBayDoorResponse  Why will you not open the pod bay door   
78           SelfAware        Can you prove you are self-aware   
79           SelfAware        Can you prove you are self aware   
80           SelfAware      Can you prove you have a conscious   

                                             response  
0          Hi human, please tell me your GeniSys user  
1       Hello human, please tell me your GeniSys use

In [26]:
dataset.duplicated

<bound method DataFrame.duplicated of                 intent                                    text  \
0             Greeting                                      Hi   
1             Greeting                                Hi there   
2             Greeting                                    Hola   
3     GreetingResponse                         My user is Adam   
4     GreetingResponse                            This is Adam   
..                 ...                                     ...   
76  PodBayDoorResponse   Why can you not open the pod bay door   
77  PodBayDoorResponse  Why will you not open the pod bay door   
78           SelfAware        Can you prove you are self-aware   
79           SelfAware        Can you prove you are self aware   
80           SelfAware      Can you prove you have a conscious   

                                             response  
0          Hi human, please tell me your GeniSys user  
1       Hello human, please tell me your GeniSys user  
2

Cosine Distance for Similarity of Texts

In [27]:
def cosine_distance_countvectorizer_method(s1, s2):

    # sentences to list
    allsentences = [s1 , s2]

    # packages
    from sklearn.feature_extraction.text import CountVectorizer
    from scipy.spatial import distance

    # text to vector
    vectorizer = CountVectorizer()
    all_sentences_to_vector = vectorizer.fit_transform(allsentences)
    text_to_vector_v1 = all_sentences_to_vector.toarray()[0].tolist()
    text_to_vector_v2 = all_sentences_to_vector.toarray()[1].tolist()

    # distance of similarity
    cosine = distance.cosine(text_to_vector_v1, text_to_vector_v2)
    return round((1-cosine),2)

Finding Response

In [28]:
def respond(text):
    maximum = float('-inf')
    response = ""
    closest = ""
    for i in dataset.iterrows():
        sim = cosine_distance_countvectorizer_method(text, i[1]['text'])
        if sim > maximum:
            maximum = sim
            response = i[1]['response']
            closest = i[1]['text']
    return response

In [35]:
while True:
    text = str(input("Input: "))
    if text.lower() == "exit":
        print("Response: Exiting.....")
        break
    print("Response:",respond(text))

Input: hello there
Response: Hello human, please tell me your GeniSys user
Input: How are you ? My user is Adam
Response: Great! Hi <HUMAN>! How can I help?
Input: What is your name ?
Response: You can call me Geni
Input: What is your real name ?
Response: My name is GeniSys
Input: Can you tell me if you are self-aware ?
Response: That is an interesting question, can you prove that you are?
Input: exit
Response: Exiting.....
